In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 15 14:29:44 2019

@author: stefgarasto
"""


import os
import pickle
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from operator import itemgetter
import time
from fuzzywuzzy import process
import itertools

In [ ]:
# flatten lists of lists
def flatten_lol(t):
    return list(itertools.chain.from_iterable(t))
flatten_lol([[1,2],[3],[4,5,6]])

In [ ]:
def print_elapsed(t0_local, task = 'current task'):
    print('Done with {}. Elapsed time: {:4f}'.format(task,time.time()-t0_local))

In [ ]:
# for spelling correction
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('/Users/stefgarasto/Google Drive/Documents/data/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
####### Definitions
def unique(list1):

    # intilize a null list
    unique_list = []
    repeated_elems = []
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
        else:
            repeated_elems.append(x)
    return unique_list, repeated_elems

In [ ]:
# make everything lowercase
def make_lowercase(x):
    if isinstance(x,str):
        return x.lower().replace('â€™','').replace('â','')
    else:
        return x

# Load and clean metadata

In [ ]:
#### get the file where the metadata is stored
metadata_dir = '/Users/stefgarasto/Google Drive/Documents/data/NOS_meta_data/'
metadata_file = 'PublishedNos Details.xlsx'
metadata_file_old = 'PublishedNos-old Details.xlsx'


In [ ]:
metadata_old = pd.read_excel(os.path.join(metadata_dir, metadata_file_old))
metadata_old.head()

In [ ]:
print(len(metadata_old), len(metadata_old['URN'].value_counts()))
ix = np.where(metadata_old['URN'].value_counts()==1)[0][0]
repeated_urns_old = metadata_old['URN'].value_counts()[0:ix].index.values
print('Number of repeated old URNs is: ',len(repeated_urns_old))
print('Number of NAN old URNs is: ',metadata_old['URN'].isnull().sum())

In [ ]:
metadata = pd.read_excel(os.path.join(metadata_dir, metadata_file))
metadata.head()

In [ ]:
print(len(metadata), len(metadata['URN'].value_counts()))
ix = np.where(metadata['URN'].value_counts()==1)[0][0]
repeated_urns = metadata['URN'].value_counts()[0:ix].index.values
print('Number of repeated old URNs is: ',len(repeated_urns))
print('Number of NAN old URNs is: ',metadata['URN'].isnull().sum())

In [ ]:
# concatenate all metadata
metadata_old.set_index = range(len(metadata),len(metadata)+len(metadata_old))
metadata = pd.concat((metadata,metadata_old),ignore_index = True,sort=False)
metadata.columns

In [ ]:
# eliminate rows with no information
metadata = metadata[~metadata['NOS Title'].isnull()]
# known bug
metadata['Name'] = metadata['Name'].map(lambda x: x.strip())
# remove duplicates (keep the last one because it has the SOC code, at least here)
ix = np.where(metadata['Name'].value_counts().values==1)[0][0]
repeated_meta_names = metadata['Name'].value_counts()[0:ix].index.values
repeated_meta_counts = metadata['Name'].value_counts()[0:ix].values
metadata.drop_duplicates(subset = 'Name', inplace = True, keep = 'last')
metadata = metadata.reset_index(drop = 'True')
metadata.columns

In [ ]:
print(len(metadata), len(metadata['URN'].value_counts()))
ix = np.where(metadata['URN'].value_counts()==1)[0][0]
repeated_urns = metadata['URN'].value_counts()[0:ix].index.values
print('Number of repeated URNs is: ',len(repeated_urns))
print('Number of NAN URNs is: ',metadata['URN'].isnull().sum())

In [ ]:
def meta_suites_corrections(x):
    if isinstance(x, str):
        x = x.strip().replace('\n','').replace('–','-').lower().replace('&','and').replace('-',' ').replace('/',' ')
        x = x.replace('(','').replace(')','').replace(':','').replace('\'','')
        x = x.replace('＆','and')
        x = ' '.join(x.split()).replace('trades','trade').replace('professionals','professional')
        x = x.replace('turbines','turbine').replace('para professional','paraprofessional')
        x = x.replace('orderivatives','or derivatives').replace('productions','production')
        x = x.replace('securities derivatives', 'securities or derivatives').replace('persons','person')
        x = x.replace('minerals','mineral')
        x = x.replace('managemnet','management')
        x = x.replace('monitorin operations','monitoring operations')
        x = x.replace('digitial','digital')
        x = x.replace('pre cast','precast')
        x = x.replace('fenestration i','fenestration, i')
        x = x.replace('managmement','management')
        x = x.replace('auxillary','auxiliary')
        x = x.replace('installations','installation')
        x = x.replace('vererinary','veterinary')
        x = x.replace('managmement','management')
        x = x.replace('wood machining','woodmachining')
        x = x.replace('mangement','management')
        x = x.replace('small holders','smallholders')
        x = x.replace('digitial','digital')
        x = x.replace('performanceandnbsp','performance')
        x = x.replace('administration nos','administration')
        x = x.replace('installing domestic fascia, soffit,', 'installation of domestic fascias, soffits')
        x = x.replace('skills for security essential employability nos','skills for security essential employability')
        x = x.replace('fenestration and installation surveying','fenestration, installation and surveying')
        # ('nail services', 100), ('mail services', 92)
        #x = x.replace('medicinal','medical')
        #x = correction(x)
    else:
        print(x)
    return x

In [ ]:
for col in ['Suite','Validity','Indicative Review Date','NOS Document Status']:
    #plt.figure()
    #sns.countplot(metadata_old[col])
    print(col,len(metadata[col].value_counts()))
    
# some suites are lists separated by ";": extract all unique suites
all_ground_suites = []
for row in metadata.index:
    if isinstance(metadata['Suite'].loc[row],str):
        all_ground_suites += metadata['Suite'].loc[row].split(";")
t0 = time.time()
all_ground_suites = [meta_suites_corrections(t) for t in all_ground_suites]
print_elapsed(t0,'correcting ground truth suites')

all_ground_suites = list(set(all_ground_suites))
if all_ground_suites[0]=='':
    all_ground_suites = all_ground_suites[1:]
print('Number of unique suites from Active IS metadata: ',len(all_ground_suites))

In [ ]:
CHECK_SUITES = False
if CHECK_SUITES:
    counter = 0
    t0 = time.time()
    for ix in range(len(all_ground_suites)):
        out = process.extract(all_ground_suites[ix], all_ground_suites, limit=2)
        if out[1][1] > 88 and out[1][1]<100:
            #if str(set(out[0][0])-set(out[1][0])).isdigit():
            print(out)
            counter+=1
    print_elapsed(t0,'listing almost-the-same suites')

In [ ]:
# now, split, clean and rejoin the suites
def clean_meta_suites(x):
    if isinstance(x,str):
        x = ";".join([meta_suites_corrections(t) for t in x.split(';')])
    return x
metadata['Suite'] = metadata['Suite'].map(clean_meta_suites)

In [ ]:
# finally, make it all lowercase
metadata = metadata.applymap(make_lowercase)

In [ ]:
metadata_old = None

In [ ]:
print('There are {} rows in the metadata with non-null SOC code'.format(metadata['SOC Code'].isnull().sum()))

In [ ]:
# TODO: for NOS CAtegory, change jobspecific to jobspecific
metadata['NOSCategory'][metadata['NOSCategory']=='jobspecific'] = 'job specific'

# Load and clean data extracted from pdfs

In [ ]:
######## Load and concatenate all NOS

data_dir = '../../results/NOS/extracted'
figure_dir = '../../results/NOS/nlp_analysis'
which_files = ['extracted_standards_New NOS 1.pickle', 'extracted_standards_New NOS 2.pickle']
for ii in range(1,23):
    which_files.append('extracted_standards_Old NOS {}.pickle'.format(ii))
#which_file = 'extracted_standards_Old NOS 22.pickle'

repeated_refs = []
tot_count = 0
# load all the extracted NOS and concatenate them
for ii,which_file in enumerate(which_files):
    version = 'new_' #'v2_'
    print(version + which_file)
    with open(os.path.join(data_dir, version + which_file),'rb') as f:
        standard_info_partial, standard_ref_partial, failed, _ = pickle.load(f)
    tot_count += len(standard_ref_partial)
    # quick check that the list of keys in the standard dictionary is the
    # same as the URNs in the standard_ref list
    tmp = [k.replace('_v2','').replace('_v3','').replace('_v4','') for k in standard_info_partial.keys()]
    if not unique(standard_ref_partial)[0] == unique(tmp)[0]:
        print('Dictionary keys and standard refs do not correspond. Something is wrong.')
    if ii == 0:
        standard_info= standard_info_partial
        standard_ref = standard_ref_partial
    else:
        # check if there is any ref that is already in the full list
        ref_intersection= list(set(standard_ref_partial).intersection(set(standard_ref)))
        if len(ref_intersection):
            #keep track of the repeated ones and add _v followed by the next 
            # available number to their dict keys
            repeated_refs = repeated_refs + ref_intersection
            for ref in ref_intersection:
                new_version = 2
                while ref + '_v{}'.format(new_version) in list(standard_info_partial.keys()):
                    new_version+=1
                    #print(new_version)
                standard_info_partial[ref + '_v{}'.format(new_version)
                    ] = standard_info_partial.pop(ref)
        standard_info.update(standard_info_partial)
        standard_ref = standard_ref + standard_ref_partial


standard_info_partial = None
standard_ref_partial = None
print(len(standard_info),tot_count)

# NOTE: The total number of NOS is 22757

In [ ]:
# check if it's fine to turn some of the fields into string
# Specifically, these fields should be strings: URN, Title, Developed_by, Date_approved
# Indicative review date, Validity, Status, Originating organisation, Suite
# OBS:I would keep the Original URN as a list, since it can contain multiple one
# How about keywords?
# Also, version number should be an integer
for idct, key_dct in enumerate(standard_info):
    dct = standard_info[key_dct]
    needed_sections = []
    for key in ['Developed_by','Date_approved', 'Indicative_review_date', 
                'Originating_organisation', 'Status','Suite', 'Validity','Version_number']:
        if key in dct.keys():
            if len(dct[key]):
                tmp = dct[key][0]
                for ii in range(1,len(dct[key])):
                    tmp += dct[key][ii]
                dct[key] = tmp
                if key=='Version_number':
                    # sometimes, the Version number contains spurious characters, not only digit
                    # so I need to dig out whether there is a digit in it and take it out
                    # yes: sometimes this can return the wrong value, because it will stop at the
                    # first digit it encounters
                    s = dct['Version_number']
                    re_result = re.search('\d{1,2}',s) #('\d+',s)
                    if re_result:
                        dct['Version_number'] = int(s[re_result.span()[0]:re_result.span()[1]])
                    else:
                        dct['Version_number'] = -1
            else:
                # in this case the field was an empty list, meaning the section was empty.
                if key=='Version_number':
                    dct['Version_number'] = -1
                else:
                    dct[key] = 'empty'
    

    #int(float(dct['Version_number']))
print('Done')

In [ ]:
DROP = False
qualifier = 'new_no_dropped'


In [ ]:
#%% Turn the nested dictionary into a dataframe
nos_data = pd.DataFrame.from_dict(standard_info, orient = 'index')
print(nos_data.columns)
# first, clean the data
# find all the rows with at least one column == 'empty' or == -1 (see above as to why)
# also, there are some bad URNs (see commented cell below), which are:
bad_urns = ['{Unique Reference Number]']
nos_data['empty'] = nos_data.apply(lambda x: (x=='empty') | (x==-1)).any(axis=1)
print('Number of rows we might need to drop is: ', np.sum(nos_data['empty']))
dropped_urns = list(nos_data.index[nos_data['empty']])
dropped_filenames = list(nos_data['file_name'][nos_data['empty']])
dropped_titles = list(nos_data['Title'][nos_data['empty']])
# try not eliminating the rows
if DROP:
    nos_data_clean = nos_data[~nos_data['empty']]
else:
    nos_data_clean = nos_data
    

In [ ]:
SAVEFIG = False

In [ ]:
nos_data_clean = nos_data_clean.applymap(make_lowercase)

In [ ]:
'''
# plot histograms after the cleaning, but before any other change
# just by dropping the 630 rows above, the following histograms look much cleaner
for col in ['Originating_organisation', 'NOS Document Status', 'Validity',
       'Indicative Review Date', 'Date_approved', 'Developed By']:
    plt.figure(figsize = (5,10))
    tmp = nos_data_clean[col].value_counts()
    N = min((len(tmp),40))
    tmp[0:N].plot('barh')
    plt.xlabel(col)
    if SAVEFIG:
        plt.savefig(os.path.join(figure_dir, 'Most_common_{}_{}.svg'.format(col,qualifier)))
# without dropping the rows, we might need to sort out the status column
'''
print('not needed')

In [ ]:
# add the year in which it was approved and the indicative review year
# first you need to look for the month and the year
def extract_monthyear(x):
    reg_exp = 'jan|january|february|march|april|may|june|july|august|september|october|november|december'
    p = re.compile(reg_exp)
    # search for the regular expression and only get the first occurrence
    # we will be assuming that the year is right after the month
    if isinstance(x,str):
        # first check if they are all digits:
        if all([d.isdigit() for d in x]):
            return x
        if p.search(x.lower()):
            start, end = p.search(x.lower()).span()
            # get intermediate string:
            tmp_s = x[start:end+10]
            # find first and last digit in the intermediate string
            digits = [ii for ii,d in enumerate(tmp_s) if d.isdigit()]
            if len(digits):
                year = tmp_s[digits[0]:digits[-1]+1]
            else:
                return 'july 2130'
            # check if there is another string:
            x2 = x[end:]
            if p.search(x2.lower()):
                start2, end2 = p.search(x2.lower()).span()
                # get intermediate string:
                tmp_s2 = x2[start2:end2+10]
                # find first and last digit in the intermediate string
                digits2 = [ii for ii,d in enumerate(tmp_s2) if d.isdigit()]
                year2 = tmp_s2[digits2[0]:digits2[-1]+1]
                if int(year2)>int(year):
                    return x2[start2:end2].lower() + ' ' + year2
                else:
                    x[start:end].lower() + ' ' + year
            else:
                return x[start:end].lower() + ' ' + year
        else:
            return 'july 2130'
    else:
        return 'july 2130'

In [ ]:
extract_monthyear('2002')

In [ ]:
# locate the month for the indicative review date, turn into a datetime object, extract the year
nos_data_clean['Indicative_review_year'] = pd.to_datetime(
    nos_data_clean['Indicative_review_date'].map(extract_monthyear),errors= 'coerce',infer_datetime_format=True).map(
    lambda x: x.year)

# do the same for the date approved field
nos_data_clean['Date_approved_year'] = pd.to_datetime(
    nos_data_clean['Date_approved'].map(extract_monthyear),errors= 'coerce',infer_datetime_format=True).map(
    lambda x: x.year)

# get the rows without a good year string, to drop them in future
no_review_year_rows = list(nos_data_clean.index[nos_data_clean['Indicative_review_year']==2030.0])
no_review_year_files = list(nos_data_clean['file_name'][nos_data_clean['Indicative_review_year']==2030.0])
no_review_year_titles = list(nos_data_clean['Title'][nos_data_clean['Indicative_review_year']==2030.0])
no_approved_year_rows = nos_data_clean.index[nos_data_clean['Date_approved_year']==2030.0]
no_approved_year_files = list(nos_data_clean['file_name'][nos_data_clean['Date_approved_year']==2030.0])
no_approved_year_titles = list(nos_data_clean['Title'][nos_data_clean['Date_approved_year']==2030.0])


In [ ]:
# plot the value counts as an histogram
# indicative review data
plt.figure(figsize = (10,4))
nos_data_clean['Indicative_review_year'].value_counts().plot('bar')
#nos_data_clean['Clean Ind Review Year'].value_counts().plot('bar')
plt.xlabel('Indicative review year')
if SAVEFIG:
    plt.savefig(os.path.join(figure_dir, 'Most_common_Indicative_review_year_{}.svg'.format(qualifier)))
# Approved date
plt.figure(figsize = (10,4))
nos_data_clean['Date_approved_year'].value_counts().plot('bar')
plt.xlabel('Year approved')
if SAVEFIG:
    plt.savefig(os.path.join(figure_dir, 'Most_common_Approved_year{}.svg'.format(qualifier)))

# now joint histogram
plt.figure()
g = sns.jointplot("Date_approved_year", "Indicative_review_year", data=nos_data_clean)
if SAVEFIG:
    plt.savefig(os.path.join(figure_dir, 'Indicative_review_year_vs_Approved_year_{}.svg'.format(qualifier)))

In [ ]:
#TODO: clean up rather than delete suites, also remove years from suites but keep the "suite number"
# things to do to clean up suites names:
'''
1. remove brackets
2. traderepair --> trade repair
3. & to and
4. check why sound and sound are split
5. check wood constructions
6. eliminate years
7. plastering (construction), roofing occupations (construction) --> split. 
7a. Generally, split by comma too (not sure, actually)
8. offshore s1971 elected safety representatives is the only one with a number in the middle
8a. joking, there is also h2s & other hazardous gases; 1132 marketing and sales managers; 
9. remove - and similar (like hospitality â€“ generics)
10. be careful with laboratory and associated technical activities suite 32010 (and 42019 and 22010)
11. be careful of it users 6.2 too
12. constructionindustry
13. operationscoating
24. remove leading and end spaces
'''
def clean_suites(x):
    if isinstance(x, str):
        x = x.replace('(',' ')
        x = x.replace(')',' ')
        if ('22010' in x) or ('32010' in x) or ('42010' in x):
            x = x[0:-4]
        x = x.replace('&','and')
        # remove year
        #if 's1971' not in x:
        #    x = re.sub('\d{4}',' ', x)
        #    if 'h2s' not in x:
        #        # add a semicolon after a number so that you can split by it later
        #        x = re.sub('(\d)', r'\1;', x)
        x = x.replace('&', 'and ')
        x = x.replace('â€“', '')
        #x = x.replace('– ',' ')
        x = x.replace('–',' ')
        #x = x.replace('- ',' ')
        x = x.replace('-',' ')
        x = x.replace('/' ,' ')
        x = x.replace('.',' ')
        # eliminate multiple spaces
        x = re.sub(r'\s+', ' ', x)
        # remove leading and trailing spaces
        x = x.strip()
        #TODO: with few exception if I find a group of max 3 digits not at 
        #the end and followed by a space or a letter, that's a point I should split
        if ('wood occupation' in x) and ('adv' not in x):
            x = 'wood occupations construction'
        x = x.replace('traderepair', 'trade repair')
        x = x.replace('constructionindustry','construction industry')
        x = x.replace('operationscoating','operations coating')
        x = x.replace('ofelectrical','of electrical')
        x = x.replace('mineralsprocessing','minerals processing')
        x = x.replace('differentcountries','different countries')
        x = x.replace('combinedworking','combined working')
        x = x.replace('localgovernment', 'local government')
        x = x.replace('eventsecurity','event security')
        x = x.replace('cropproduction','crop production')
        x = x.replace('customerservices','customer services')
        x = x.replace('occupationsresidential','occupations residential')
        x = x.replace('systemsmaintenance','systems maintenance')
        x = x.replace('andrefurbishment','and refurbishment')
        x = x.replace('materialsprocessing','materials processing')
        x = x.replace('thebuilding','the building')
        x = x.replace('managerfire','manager fire')
        x = x.replace('formanagement','for management')
        x = x.replace('alliedoperation','allied operation')
        x = x.replace('safetyrequirementsfabrication','safety requirements fabrication')
        x = x.replace('operationsdrilling', 'operations drilling')
        x = x.replace('operationsbulk', 'operations bulk')
        x = x.replace('andmanagement', 'and management')
        x = x.replace('artsmanagement','arts management')
        x = x.replace('pensions','pension')
        x = x.replace('extrusion furniture', 'extrusion and furniture')
        x = x.replace('ofconstruction','of construction')
        x = x.replace('2008aeronautical', '2008 aeronautical')
        x = x.replace('2rail', '2 rail')
        x = x.replace('2006engineering', '2006 engineering')
        x = x.replace('1132 marketing','marketing')
        x = x.replace('nail services 2009 2009', 'nail services 2009')
        x = x.replace('roadbuilding','road building')
        x = x.replace('seniorroles','senior roles')
        x = x.replace('511con po1561', '')
        x = x.replace('nailservices','nail services')
        x = x.replace('servicesnos','services nos')
        x = x.replace('sitelogistics','site logistics')
        x = x.replace('securityoperations','security operations')
        x = x.replace('andtendering','and tendering')
        x = x.strip()
    return x

# this function cleans the pdf extracted suites
#nos_data_clean['Clean_suite'] = nos_data_clean['Suite'].map(meta_suites_corrections).map(clean_suites)
nos_data_clean['Clean_suite'] = nos_data_clean['Suite'].map(clean_suites)
print('Done')

In [ ]:
nos_data_clean[['Clean_suite','Suite']].sample(n=20)

In [ ]:
### Perform some more manual cleanup
# for the Validity, Status, Developed_by columns, make all lower case and drop some
# obviously wrong columns (see histograms above)
def lower_or_empty(x):
    if isinstance(x,str):
        y= x.lower()
    else:
        y= 'empty'
    # also, fix some common misspellings:
    if y in ['orginal', 'origianl','orignal','original ??????????','original3']:
        y = 'original'
    elif y in ['import', 'imported from ento', 'amended form imported ento unit']:
        y = 'imported'
    elif y == 'tailored from ento':
        y = 'tailored'
    elif y in ['currentl','curent']:
        y = 'current'
    elif y == 'may 2016':
        y = 'empty'
    elif y == 'building information modelling':
        y== 'empty'
    elif y in ['proskills uk', 'manual, electronic systems, process warning enunciatorsprocess documents and logs, checklistssafety notice boards, safety data sheets, warning signsproskills']:
        y = 'proskills'
    elif y in ['summitskillsversionnumber1', 'summit skills']:
        y = 'summitskills'
    elif y == 'manufacturersâ€™ instructions6method statements7product worksheetsengineering construction industry training board':
        y = 'empty'
    elif y == 'how signals can enter, route through and exit mixing and recordinghow to identify common signal routing stages of mixing and recordingthe relationship between the stereo master section and a singlechannel -describetypes of professional mixing and recording consolesthe features of different professional recording consolesthe types of console different music and sound sectors requirethe main sector manufacturers of equipment for studio recording andmixing studios/programming consolesthe main sector manufacturers of equipment forbroadcast/tv/av/radio consolesthe main sector manufacturers of equipment for live mobile recording,theatre/performance and live sound/pa equipment for venuescreative & cultural skills':
        y = 'creative and cultural skills'
    elif y in ['skiillsactive','skills active', 
              'skillsactiveevelop your tactical skills to achieve excellence in your sport']:
        y = 'skillsactive'
    elif y in ['cfa', 'cfa business skill @ work', 'cfa business @ work',
              'these business enterprise units may be relevant when you are setting up ordeveloping a business:ys1 explore your own business motivesys2 check your ability to run your businessys3 improve your business skillsee3 make deals to take your business forwardcfa business skills @ workz0g.docx']:
        y = 'cfa business skills @ work'
    elif y in ['mpqcrovide leadership in the mine',
              'subsequent legislation that supersedes it, as appropriate to the workactivitympqclast to specification',
              'resources: tools, equipment, utilities or services, consumablesmpqc',
              'mpqceparate and dispose of wastes and by-products from mining and relatedprocessing activities',
              'to the workplace, work activity and the associated environmental impactsmpqconitor and maintain environmental conditions in your area of responsibility',
              'mpqcrocess materials to specification','[mpqc']:
        y = 'mpqc'
    elif y in ['6.46.5semtacomputer-based recordelectronic mail',
               'complete the relevant paperwork, to include one from the following andpass it to the appropriate people:7.1 build records7.2 computer records7.3 job cards7.4 aircraft service/flight log other specific recording methodsemta']:
        y = 'semta'
    elif y == 'people1st':
        y = 'people 1st'
    elif y in ['security threatse-skills uk',
               'k12.5 verifying the accuracy, currency, completeness and relevance ofinformation created, collected, used and documented during datadesign activitiese-skills uk']:
        y = 'skills uk'
    elif y == '[habia':
        y = 'habia'
    # finally, change & to and
    y = y.replace('&','and')
    return y
    

for col in ['Status', 'Validity', 'Developed_by']:
    nos_data_clean[col] = nos_data_clean[col].map(lower_or_empty)
print('Done')

In [ ]:
# just change the incredibly long one in the validity field
tmp = [nos_data_clean['Validity'].value_counts().index][0]
tmp = [t for t in tmp if len(t)> 100]

if len(tmp):
    tmp = tmp[0]
    for row in nos_data_clean.index:
        if nos_data_clean['Validity'].loc[row] == tmp:
            nos_data_clean['Validity'].loc[row] = 'current'
            print(row)
            print(nos_data_clean['file_name'].loc[row])
            print(nos_data_clean['Title'].loc[row])

for col in ['Status', 'Validity']:
    plt.figure(figsize = (5,5))
    tmp = nos_data_clean[col].value_counts().plot('barh')
    plt.ylabel(col)
    if SAVEFIG:
        plt.savefig(os.path.join(figure_dir, 'Most_common_{}_{}.svg'.format(col,qualifier)))
    
for col in ['Developed_by']:
    plt.figure(figsize = (5,15))
    tmp = nos_data_clean[col].value_counts().plot('barh')
    plt.ylabel(col)
    if SAVEFIG:
        plt.savefig(os.path.join(figure_dir, 'Most_common_{}_{}.svg'.format(col,qualifier)))
    #print(nos_data_clean[col].value_counts())
# TODO: get empty rows

In [ ]:
'''
# Now, remove the other empty/faulty rows that were identified before.
nos_data_clean.loc[:,'empty'] = False
tmp = nos_data_clean.applymap(lambda x: (x=='empty') | (x==['empty'])).any(axis=1)
nos_data_clean.loc[:,'empty'] = tmp
empty_rows = list(nos_data_clean.index[nos_data_clean['empty']])
# include empty review/approved date in rows to remove or not
INCLUDE_YEARS = False
if INCLUDE_YEARS:
    # this removes 514 rows
    rows_to_remove = list(set(empty_rows + not_string_suites_rows + list(no_review_year_rows) + 
                              list(no_approved_year_rows)))
else:
    # this remove 17 rows
    rows_to_remove = list(set(empty_rows + not_string_suites_rows))
print('Number of rows that I would drop is: ', len(rows_to_remove))
for row in rows_to_remove: #this for loop is incredibly slow for some reason, just FYI
    nos_data_clean.loc[row,'empty'] = True
dropped_urns = dropped_urns + rows_to_remove
dropped_filenames = dropped_filenames + list(nos_data_clean['file_name'][nos_data_clean['empty']])
dropped_titles = list(nos_data_clean['Title'][nos_data_clean['empty']])
# again, I'm not actually deleting rows this time
if DROP:
    nos_data_cleaner = nos_data_clean[~nos_data_clean['empty']]
else:
    nos_data_cleaner = nos_data_clean
'''
print('not needed')

In [ ]:
# TODO: NLP analysys. The final output dataset is called nos_data_cleaner
nos_data_clean.columns

# Join pdf-extracted data and the metadata from ActiveIS

Note that both datasets should have already been cleaned. However, some leading and trailing spaces might still appear when we split the suites by ";". Also, I still don't know how to split lists of suites from the pdf-extracted data that are not actually separated by ";" (TODO)

In [ ]:
print('Number of NOS extracted from pdfs is {}'.format(len(nos_data_clean)))
print('Number of NOS for whom we have metadata is {}'.format(len(metadata)))

In [ ]:
# some filenames appear different but they actually do have a match: find that match by using the URN
t0 = time.time()
missings = list(set(metadata['Name'])-set(nos_data_clean['file_name']))
# + len(set(nos_data_clean['file_name'])-set(metadata['Name']))
counter = 0
for ix in missings:
    row = metadata[metadata['Name']==ix]
    urn = row['URN'].values
    if len(urn)==1:
        row2 = nos_data_clean[nos_data_clean['URN']==urn[0]]
        if len(row2):
            counter += 1
            ixx = row2.index[0]
            nos_data_clean['file_name'].loc[ixx] = ix
print(counter,len(missings),'{:.4f}'.format(time.time()-t0))

In [ ]:
recovered = counter
metadata_missing = len(list(set(metadata['Name'])-set(nos_data_clean['file_name'])))
intersected = set(metadata['Name']).intersection(set(nos_data_clean['file_name']))
print('Number of NOS matched from metadata to pdf-extracted is {}'.format(len(list(intersected))))
print('Number of metadata NOS not matched is {}'.format(metadata_missing))
print('Number of pdf-extracted NOS not matched is {}'.format(
    len(list(set(nos_data_clean['file_name']) - set(metadata['Name'])))))

In [ ]:
nos_data = nos_data_clean.set_index('file_name').join(metadata.set_index('Name'), lsuffix = '_2') 
tmp = nos_data.isnull().sum()
print('Missing values from the joined dataframe: ')
for col in nos_data.columns.sort_values():
    print(col,'-',tmp[col])
print(len(nos_data)-len(metadata)+523-252)

In [ ]:
nos_data[['Clean_suite','Suite']].sample(n=10)

In [ ]:
# replace the existing metadata when it's null
cols2copy = ['Developed By', 'NOS Title', 'Occupations', 
       'Validity', 'NOS Document Status', 'URN',
       'Keywords', 'Original URN', 'Suite']
backup_cols = ['Developed_by', 'Title', 'Relevant_occupations',
       'Validity_2', 'Status', 'URN_2',
        'Keywords_2', 'Original_URN', 'Clean_suite']
for ix,col in enumerate(cols2copy):
    nullrows = nos_data[col].isnull()
    nos_data[col].loc[nullrows] = nos_data[backup_cols[ix]].loc[nullrows]
nos_data['Suite'][nos_data['Suite'].isnull()] = 'others'
# The year needs to be dealt with separately
# Indicative Review Date - 1029
# Indicative_review_date - 260

In [ ]:
nos_data.isnull().sum()

In [ ]:
# find repeated file names, URNs and NOS titles
# store repeated indices (that is, file names) - I already removed duplicates from the metadata, so A inherited all
# possible duplicates from nos_data
ix = np.where(nos_data.index.value_counts().values==1)[0][0]
repeated_names = list(nos_data.index.value_counts()[0:ix].index.values)
repeated_names_counts = list(nos_data.index.value_counts()[0:ix].values)
repeated_names += list(repeated_meta_names)
#repeated_names_counts += repeated_meta_names_counts
# now URN
col = 'URN'
ix = np.where(nos_data[col].value_counts().values==1)[0][0]
repeated_urns = nos_data[col].value_counts()[0:ix].index.values
repeated_urns_counts = nos_data[col].value_counts()[0:ix].values

col = 'NOS Title'
ix = np.where(nos_data[col].value_counts().values==1)[0][0]
repeated_titles = nos_data[col].value_counts()[0:ix].index.values
repeated_titles_counts = nos_data[col].value_counts()[0:ix].values

# now, drop rows for whom both URN and Title are exactly the same
B = nos_data[nos_data.duplicated(subset= ['NOS Title','URN'])]

# save them for another time
with open(os.path.join(figure_dir,'repeated_rows.pickle'),'wb') as f:
    pickle.dump((repeated_names,repeated_names_counts,repeated_urns,repeated_urns_counts,repeated_titles,
               repeated_titles_counts,B,['repeated_names,repeated_names_counts,repeated_urns,repeated_urns_counts,'+
               'repeated_titles,repeated_titles_counts,Titles+URNs duplicated']), f)


In [ ]:
# now, drop the duplicates indices
nos_data = nos_data.groupby(nos_data.index).first()
## now, drop rows that have both URN and Title exactly the same
#B = A.drop_duplicates(subset = ['NOS Title','URN'])
#len(B),len(A)

In [ ]:
# extract the year from the metadata information. As expected, lots of them have the year 1905
nos_data['Indicative Review Year'] = pd.to_datetime(nos_data['Indicative Review Date'],
                                                         errors = 'coerce').map(lambda x: x.year)

# for the rows with year = 1905.0 or that are null it's better to use the extracted ones
nos_data['Clean Ind Review Year'] = nos_data['Indicative Review Year']
nos_data['Clean Ind Review Year'].loc[nos_data['Indicative Review Year']==1905.0] = nos_data[
    'Indicative_review_year'].loc[nos_data['Indicative Review Year']==1905.0]

# also replace the rows with a null value with the extracted year
nullrows = nos_data['Clean Ind Review Year'].isnull()
nos_data['Clean Ind Review Year'].loc[nullrows] = nos_data['Indicative_review_year'].loc[nullrows]

In [ ]:
# Analyse the suite fields to check for possible outliers. Specifically, I want to find a length threshold to discard
# spurious elements (some might be mistakes, some might be multiple suites combined into one)

# One suite fields, can contain multiple suites split by ';'. Perform the split and
# get the lenght of each sub-element

suite_len = []
all_suites = []
not_string_suites_rows = []
not_string_suites_titles = []
not_string_suites_names = []
col2use = 'Suite'
for row in nos_data.index:
    if isinstance(nos_data[col2use].loc[row],str):
        y = nos_data[col2use].loc[row].split(';')
        y = [t.lstrip().rstrip() for t in y]
        suite_len += [len(t) for t in y]
        all_suites += y
    else:
        not_string_suites_rows_names.append(row)
        not_string_suites.append(nos_data['URN'].loc[row])
        not_string_suites_titles.append(nos_data['Title'].loc[row])
        

# get the unique list of suites
unique_suites = list(set(all_suites))
unique_len = [len(t) for t in unique_suites]
sort_ind = np.argsort(unique_len)
# order them by length
unique_len = itemgetter(*sort_ind)(unique_len)
unique_suites = itemgetter(*sort_ind)(unique_suites)

# now print all the suites that contain more than N characters: how many can we safely disregard? Is there a
# threshold that separates good from bad ones:
N = 90
# collect the indices of unique suites that are longer than N characters
tmp = [ t for t,tval in enumerate(unique_len) if tval>N]
for t in tmp:
    print(unique_len[t],unique_suites[t])
    print('\n')
    
# Observation: if I select threshold = 90, I shouldn't be keeping any "bad" suites, but I'll be discarding some good
# ones. There are, I think, 4 good and 3 bad ones that are between 90 and 122 characters. 
# DECISION: Let's set the threshold to 125
print(not_string_suites_rows)

In [ ]:
# cycle again through the dataframe to keep the whole suites list, have one extra column with only the first suite in 
# each list (not ideal, I know, but then I can categorise all the rows), and eliminate the sub-elements that are
# longer than Nth characters. Note that for rows with only one suite to begin with, nothing should change

def get_all_and_single_suites(Nth,nos_data_clean,col):
    not_string_suites_rows = []
    not_string_suites_titles = []
    not_string_suites_names = []
    #nos_data_clean['All_suites'] = 'empty' # initialise, so that if it is not overridden it will appear as empty
    #nos_data_clean['One_suite'] = 'empty' # initialise, so that if it is not overridden it will appear as empty
    all_suites = []
    one_suite = []
    for row in nos_data_clean.index:
        if isinstance(nos_data_clean[col].loc[row],str):
            y = nos_data_clean[col].loc[row].split(';')
            y = [t for t in y if len(t)<Nth]
            if len(y):
                y = [t.strip() for t in y]
                y = [t for t in y if len(t)]
                all_suites.append(y)
                one_suite.append(y[0])
            else:
                tmp= ''
                y = nos_data_clean[col].loc[row].split(' ')
                for t in y:
                    if t not in tmp:
                        tmp += ' ' + t
                    else:
                        break
                # now split by comma
                tmp = tmp.split(',')[0].strip()
                #print(tmp)
                all_suites.append([tmp])
                one_suite.append(tmp)
        else:
            not_string_suites_names.append(row)
            not_string_suites_rows.append(nos_data_clean['URN'].loc[row])
            not_string_suites_titles.append(nos_data_clean['Title'].loc[row])
            all_suites.append(['empty'])
            one_suite.append('empty')
    return all_suites, one_suite, not_string_suites_rows, not_string_suites_titles, not_string_suites_names

# apply the above function once for the metadata suites:
Nth = 200
col = 'Suite'
all_suites, one_suite, _, _, _ = get_all_and_single_suites(Nth,nos_data,col)
nos_data['All_suites'] = all_suites
nos_data['One_suite'] = one_suite

'''
# now for the pdf extracted suites, for an eventual comparison
Nth = 125
col = 'Clean_suite'
all_suites, one_suite, _, _, _ = get_all_and_single_suites(Nth,nos_data,col)
nos_data['All_suites'] = all_suites
nos_data['One_suite'] = one_suite
'''
print('Done')

In [ ]:
print(nos_data[[col,'All_suites','One_suite']].sample(n=10))

In [ ]:
# now flatten the lists of lists with the suites and count how many unique suites there are:
all_suites_flatten = flatten_lol(all_suites)
# compare with the numbers from the ActiveIS metadata
print('Unique new suites: ',len(list(set(all_suites_flatten))),'Unique ActiveIS suites: ', len(all_ground_suites))
print('\n')
tmp = set(all_suites_flatten) - set(all_ground_suites)
print('Suites ''obtained'' after joining all the data ({}):'.format(len(tmp)))
print(tmp)
print('\n')
tmp = set(all_ground_suites) - set(all_suites_flatten)
print('Suites ''lost'' after joining all the data ({})'.format(len(tmp)))
print(tmp)

In [ ]:
### Plot some summary statistics of fields content
SAVEFIG2 = False
cols2plot = ['Originating_organisation',
        'Date_approved_year', 'Developed By',
        'Validity', 'Clean Ind Review Year', 'NOS Document Status',
        'NOSCategory', 'One_suite', 'Version_number']
#        'Keywords', 'Occupations', 'Skills']
hs = [10,5,8,5,5,5,5,10,5]
ws = [18,12,18,6,12,6,6,18,9]
with sns.plotting_context('talk'):
    for ix,col in enumerate(cols2plot):
        plt.figure(figsize = (ws[ix],hs[ix]))
        tmp = nos_data[col].value_counts()
        N = min(40,len(tmp))
        tmp.iloc[:N].plot('bar')
        plt.xlabel(col)
        plt.tight_layout()
        if SAVEFIG2:
            plt.savefig(os.path.join(figure_dir, 'Most_common_{}_postjoining_{}.png'.format(col,qualifier)))


In [ ]:
# TODO: plot the SOC codes separately.
# NOTES: some are a list, some are wrong, some are less than 4 digits (only few exceptions: twentytwo 1s and one 9)
col = 'SOC Code'
with sns.plotting_context('talk'):
    plt.figure(figsize = (21,5))
    # separate groups of digits and eliminate everything that is not a digit
    flattened_socs= flatten_lol(nos_data[col].map(lambda x: re.findall(r"[\d']+", str(x))).values)
    tmp = pd.DataFrame(flattened_socs)[0].value_counts()
    tmp.iloc[0:50].plot('bar')
    plt.xlabel('SOC codes, 4 digits')
    plt.tight_layout()
    if SAVEFIG2:
        plt.savefig(os.path.join(figure_dir, 'Most_common_{}_4digit_postjoining_{}.png'.format(col,qualifier)))
    # now plot for different number of digits
    
    ws = [10,15,20]
    for ix in range(1,4):
        plt.figure(figsize = (ws[ix-1],5))
        tmp = pd.DataFrame(flattened_socs)[0].map(lambda x: x[0:ix]).value_counts()
        if ix<3:
            g = tmp.plot('bar')
            g.set_xticklabels(g.get_xticklabels(), rotation = 0)
        else:
            g = tmp.iloc[0:50].plot('bar')
        plt.xlabel('SOC codes, {} digit'.format(ix))
        plt.tight_layout()
        if SAVEFIG2:
            plt.savefig(os.path.join(figure_dir, 
                                     'Most_common_{}_{}digit_postjoining_{}.png'.format(col,ix,qualifier)))



In [ ]:
# actually clean the SOC codes
nos_data['Clean SOC Code'] = nos_data[col].map(lambda x: re.findall(r"[\d']+", str(x)), na_action = 'ignore')

In [ ]:
# plot how many NOS are in the most common and the least common suites
tmp = nos_data['One_suite'].value_counts()
print('There are {} suites when only taking the first one.'.format(len(tmp)))
print('The least common suites are (all appear only once):')
for t in tmp.index[-40:]:
    print(t)

In [ ]:
# now get the distribution of suites and originating organisations for NOS that have SOC codes
# this needs to be moved to collate_and_analyse_nos
for col in ['One_suite', 'Originating_organisation']:
    duplicated_suites = df_nos[df_nos['Clean SOC Code'].notnull()][col].value_counts()
    all_suites = df_nos[col].value_counts()
    if col == 'One_suite':
        plt.figure(figsize = (15,10))
    else:
        plt.figure(figsize = (15,7))
    with sns.plotting_context('talk'):
        duplicated_suites[0:40].plot('bar')
    plt.tight_layout()
    if SAVELSH or True:
        plt.savefig(os.path.join(output_dir,'{}_proportions_for_soc_having_nos.png'.format(col)))

    # divide by the sum (so we get fractions)
    duplicated_suites = duplicated_suites.map(lambda x: x/duplicated_suites.sum())
    all_suites = all_suites.map(lambda x: x/all_suites.sum())
    
    # get the ratio of proportions with respect to the full distribution
    suites_ratio = {}
    for row in duplicated_suites.index:
        suites_ratio[row] = duplicated_suites.loc[row]/all_suites.loc[row]
    suites_ratio = pd.DataFrame.from_dict(suites_ratio, orient = 'index', columns = ['ratio'])
    # order by decreasing ratios 
    suites_ratio= suites_ratio.sort_values(by='ratio', ascending = False)
    # plot the ratio
    if col == 'One_suite':
        plt.figure(figsize = (20,10))
        ix = list(np.arange(0,50)) #list(np.arange(0,14))
#        ix+= list(np.arange(15,51))
    else:
        plt.figure(figsize = (15,8.5))
        ix = [0,1,2,4,5,6,7,9,11,13,14,15,17,19,20,21,22,23,24,25,27,29,30,
            31,33,34,35,36,41,43,44,45,46,51,52] #[0,1,4,6,7,9,10,12,13,14,15,16,17] + list(np.arange(19,54))
#        ix+= list(np.arange(15,50))
        #print([(i,t) for i,t in enumerate(suites_ratio['ratio'].index[:55])])
    with sns.plotting_context('talk'):
        suites_ratio['ratio'].iloc[ix].plot('bar')
    plt.tight_layout()
    if SAVELSH or True:
        plt.savefig(os.path.join(output_dir,'{}_ratios_for_soc_having_nos.png'.format(col)))

### Add the full text field, and save a subset of the data (only the relevant columns)

In [ ]:
# add one columns that collate all the data from multiple sections: title, overview, keywords, 
# relevant occupations (?), knowledge, performance, scope, glossary, behaviours, skills, values.

def get_full_text(data_rows):
    t0 = time.time()
    full_texts = []
    full_texts_keywords = []
    for ii,row in enumerate(data_rows.index):
        data_row = data_rows.loc[row]
        for col in ['NOS Title', 'Knowledge_and_understanding', 
                    'Performance_criteria', 'Overview','Scope_range', 'Glossary', 
                    'Behaviours', 'Skills','Values','Occupations']: #'Relevant_occupations', #'extra_meta_info'?
            if col == 'NOS Title':
                full_texts.append(data_row[col] + ' ')
                full_texts_keywords.append(data_row[col] + ' ')
            else:
                if isinstance(data_row[col],list): #not np.isnan(data_row[col]):
                    L = len(data_row[col])
                    for l in range(L):
                        full_texts[-1] += data_row[col][l] + ' '
                        full_texts_keywords[-1] += data_row[col][l] + ' '
        # add the keywords columns only to the full_text_keywords
        if isinstance(data_row['Keywords'], list):
            L = len(data_row['Keywords'])
            for l in range(L):
                full_texts_keywords[-1] += data_row['Keywords'][l] + ' '
        # keep track of where you are
        if ii%1000 == 999:
            print('Got to row {}. Time elapsed so far is {}'.format(ii,time.time()-t0))
    return full_texts, full_texts_keywords

out, _ = get_full_text(nos_data) #.apply(get_full_text, axis = 0)
        
    

In [ ]:
# show example of full text
out[0][0:500]

In [ ]:
# add the full_text column to the dataset: TODO: check
nos_data_full = nos_data.join(pd.DataFrame(out, columns = ['full_text'], index = nos_data.index))
#nos_data_full = nos_data_full.join(pd.DataFrame(out_kw, columns = ['full_text_keywords'], 
#                                                   index = nos_data_cleaner.index))

In [ ]:

# set the relevant columns
relevant_cols = ['Title', 'NOS Title', 'URN', 'Original URN', 
                 'Overview', 'Knowledge_and_understanding', 'Performance_criteria',
                 'Scope_range', 'Glossary', 'Behaviours', 'Skills', 'Values', 'full_text',
                 'Originating_organisation', 'Date_approved', 'Date_approved_year', 
                 'Indicative Review Date', 'Clean Ind Review Year', 'Version_number',
                 'Links_to_other_NOS', 'External_Links', 'Developed By', 'Validity', 
                 'Keywords', 'Clean SOC Code', 'NOS Document Status', 'NOSCategory',
                 'Suite', 'SuiteMetadata', 'Occupations', 'OccupationsMetadata', 'One_suite', 'All_suites',
                 'notes', 'empty', 'extra_meta_info', 'Created', 'Modified', 'Item Type', 'Path']
#final_relevant_cols = relevant_cols + ['full_text']
nos_data_full.columns

In [ ]:
# save the dataset with only a subset of the columns (title, suite and full_text)
#nos_data_full.to_csv(
#    data_dir + '/all_nos_input_for_nlp.csv', columns = ['Title','full_text','Suite','All_suites','One_suite'], 
#    index = True, header = True)
#[['Title','full_text','Suite','All_suites','One_suite']]
nos_data_full[relevant_cols].to_pickle(
    data_dir + '/all_nos_input_for_nlp_postjoining_{}.zip'.format(qualifier))